In [0]:

from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


#Read file into dataFRAME

In [0]:
df1=sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load('dbfs:/FileStore/shared_uploads/sanaeelmadani3@gmail.com/ratings.csv')
df1.printSchema()
df1.show()



root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+-------

In [0]:
df2=sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load('dbfs:/FileStore/shared_uploads/sanaeelmadani3@gmail.com/movies.csv')
df2.printSchema()
df2.show()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Dr

In [0]:
df3=df1.join(df2,"movieId")
df3.show(3)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
+-------+------+------+----------+--------------------+--------------------+
only showing top 3 rows



#Data preparation

use only column data of "userId","movieID" and "rating"

In [0]:
data=df1.select("userId","movieId","rating")
data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1287|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
|     1|   1953|   4.0|
|     1|   2105|   4.0|
|     1|   2150|   3.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2455|   2.5|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
+------+-------+------+
only showing top 20 rows



divide data ,70% for training and 30% for testing

In [0]:
splits=data.randomSplit([0.7,0.3])
train=splits[0].withColumnRenamed("rating","label")
test=splits[1].withColumnRenamed("rating","truelabel")


calculate number of rows

In [0]:
train_rows=train.count()
test_rows=test.count()
print("number of training data rows:", train_rows,",number of testing data rows:",test_rows)

number of training data rows: 70382 ,number of testing data rows: 29622


#Define model and train it

Define ALS(Alternating Least Square) as our recommender system

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="label")
#train our ALS model
model = als.fit(train)
print("training is done")

training is done


#Predict testing data

In [0]:
prediction=model.transform(test)
prediction.join(df2,"movieId").select("userId","title","genres","prediction","truelabel").show(3)

+------+--------------------+--------------------+----------+---------+
|userId|               title|              genres|prediction|truelabel|
+------+--------------------+--------------------+----------+---------+
|   148|Postman, The (Pos...|Comedy|Drama|Romance| 3.9142468|      4.0|
|   148|Star Trek: Genera...|Adventure|Drama|S...| 3.9133904|      4.0|
|   148|Manhattan Murder ...|      Comedy|Mystery| 5.8522797|      3.5|
+------+--------------------+--------------------+----------+---------+
only showing top 3 rows



#Evaluate the accuracy of our model

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="truelabel",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = nan


the result is nan (not a number),this happen whene we have a user in data testing that is not available in data training to deal this we have to remove failure predictions.

In [0]:
a=prediction.count()
print("number of original data rows:" , a)
#drop rows with any missing data
cleanPrediction=prediction.dropna(how="any" , subset=["prediction"])
b=cleanPrediction.count()
print("number of rows after dropping data with missing value:" , b)
print("number of missing data",a-b)

number of original data rows: 29622
number of rows after dropping data with missing value: 28370
number of missing data 1252


In [0]:
rmse = evaluator.evaluate(cleanPrediction)
print (rmse)

1.192115194385137


# Generate top 10 movie recommendations for each user

In [0]:
userRecs = model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{5114, 9.904921}...|
|     3|[{106696, 5.77886...|
|     5|[{1946, 6.485501}...|
|     6|[{334, 8.83519}, ...|
|    12|[{44761, 6.976275...|
|    13|[{3088, 7.8802853...|
|    15|[{3915, 5.4019246...|
|    16|[{1289, 7.8827085...|
|    19|[{2303, 6.0462775...|
|    20|[{3973, 8.740786}...|
|    22|[{55247, 6.00459}...|
|    26|[{2718, 6.493002}...|
|    27|[{45666, 9.074994...|
|    28|[{70336, 10.95790...|
|    31|[{3134, 5.779979}...|
|    34|[{3134, 6.259865}...|
|    37|[{3015, 8.576467}...|
|    40|[{6003, 6.889184}...|
|    41|[{2872, 6.416874}...|
|    43|[{68554, 6.988325...|
+------+--------------------+
only showing top 20 rows



#Generate top 10 user recommendations for each movie

In [0]:
movieRecs = model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     12|[{116, 11.259563}...|
|     26|[{336, 9.150837},...|
|     27|[{336, 7.8862715}...|
|     28|[{336, 9.541887},...|
|     31|[{116, 7.1490474}...|
|     34|[{259, 6.8370585}...|
|     44|[{145, 7.397416},...|
|     53|[{336, 8.81456}, ...|
|     65|[{116, 7.065312},...|
|     76|[{356, 7.58087}, ...|
|     78|[{326, 8.061222},...|
|     81|[{180, 11.739502}...|
|     85|[{259, 7.6830597}...|
|    101|[{326, 8.938707},...|
|    103|[{424, 6.998409},...|
|    126|[{310, 5.8028116}...|
|    148|[{375, 7.48441}, ...|
|    155|[{629, 7.772193},...|
|    159|[{411, 7.45333}, ...|
|    183|[{45, 7.3895845},...|
+-------+--------------------+
only showing top 20 rows

